In [1]:
# https://catboost.ai/en/docs/concepts/python-quickstart
import catboost as cb
from catboost import CatBoostClassifier


print ('catboost version', cb.__version__)

catboost version 1.0.6


# K-Fold CV for Boosting Methods Using SKLearn 

In [2]:
#Importing required libraries
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
import wfdb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd
import random
import statistics

In [3]:
import xgboost as xgb
from sklearn import metrics
from lightgbm import LGBMClassifier

import lightgbm
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 70 #display 70 dpi in Jupyter Notebook, may consider100 dpi 
plt.rcParams['savefig.dpi'] = 300 #define 300 dpi for saving figures

import seaborn as sns
## here are some settings 
sns.set_style('whitegrid')
sns.set(rc={"figure.dpi":70, 'savefig.dpi':300}) #defining dpi setting
sns.set_context('notebook')
sns.set_style("ticks")
from IPython.display import set_matplotlib_formats

### To load sklearn dataset
sklearn.datasets.load_breast_cancer(*, return_X_y=False, as_frame=False)[source]¶

Load and return the breast cancer wisconsin dataset (classification).

In [5]:
#data = load_breast_cancer(as_frame = True); 
#df = data.frame; 
#print(df.shape)
# In this block use the code we looked at on 6/30 as a reference to save our rr-intervals as a .
rlist = []
records = 'mit-bih-raw\RECORDS' # Replace the _______ with the name of the records file in your MIT-BIH data folder
with open(records) as rfile: #Then we open the file 
                             #The 'with' command only opens the file while we are in it. Automatically closes the file when we're not
    for record in rfile:  # Then we iterate through the lines in the file
        record = record[0:len(record)-1] # Remove any erronious new line characters at the end ('\n')
        rlist.append(record) # Then build an array with it
        

###### Step 1: Initialize all Arrays
             # Below, replace all of the ___ with the command that declares an array/list
             # hint: https://stackoverflow.com/questions/1514553/how-to-declare-an-array-in-python
samples = [] # will house the samples of all subjects
good_list = [] # will list the names of the subjects we successfully extracted
bad_list = [] # will house the names of the subjects we failed to extract
qrs = [] # will house the indices of R-Peaks for all subjects
atr_label = [] # will house the labels for each rhythm annotation for all subjects
atr_locs = [] # will house the locations corresponding to the rhythm annotation labels


###### Step 2: Extract Information
for x in tqdm(rlist): #this will iterate through te records that we found above
  
    try: # A try statement will run the except statement if for some reason the try commands fail
         # In this case I use the try statement because one of the subjects has no signal data causing failure
         # I then use bad_list and good_list so that all of the indices in rlist match with the arrays we initialized in Step 1, above
        ######################################################
            # Below find the wfdb function that will return the information that is described below 
        # Then replace _____ with the correct function call
        samp = wfdb.rdsamp('mit-bih-raw/'+x) # wfdb._____(file_location) will read the signal & header data and return a 2 value array
            # samp[0] - the signal data is the raw reading from the ecg. Each value is a sample taken.
            # samp[1] - the header data includes things about the signal data such as:
              # samples per section, denoted 'fs'
              # number of signals, denoted 'n_sig'
            
        ######################################################
        samples.append(samp) #add it to our array for all subject
            #What is our file extension that has the annotation we want? Find it here and replace _____ with it 
            #hint: READ THE VARIABLE NAMES!!!!
        qrs_tmp = wfdb.rdann('mit-bih-raw/'+x, extension='qrs') #extract the QRS Info
        qrs_locs = np.array(qrs_tmp.sample, dtype='int') #Get just the loccation of R-Peaks from the QRS Info
        qrs.append(qrs_locs) # Add to our array for all subjects
        
            #Do the same thing here
        atr = wfdb.rdann('mit-bih-raw/'+x,extension='atr') #extract the atr info which stores the rhythm type(s) over the whole signal
        atr_label.append(atr.aux_note) # aux_note stores the type of rhythm - main two are '(N' for normal and '(AFIB' for AFIB
        atr_locs.append(np.append(atr.sample, len(samp[0]))) #I add the length of the whole sample to the end for better visualization later
        
        good_list.append(x) # when all extraction is successful append the record name to good_list
    except Exception as exep:
        print(exep) # Alert the user of an exception
        bad_list.append(x) # add to the bad list

        

  0%|          | 0/25 [00:00<?, ?it/s]

sampto must be greater than sampfrom
sampto must be greater than sampfrom


100%|██████████| 25/25 [00:22<00:00,  1.12it/s]


In [6]:
features_df_list = []
for subject in good_list:
    data = pd.read_csv(f'subject-features-dataframes/subject{subject}.csv')
    data = data.assign(Subject = subject)
    #print(data)
    features_df_list.append(data)
#features_maindf = pd.concat(features_df_list, ignore_index = True)

In [7]:
features_maindf = pd.concat(features_df_list, ignore_index = True)
print(features_maindf.columns)

Index(['Unnamed: 0', 'AppEn', 'SampEn', 'CosEn', 'RangeEn', 'FuzzyEn',
       'EntropyAF', 'PermEn', 'BubbleEn', 'MultiscaleEn', 'Volatility',
       'Rhythm', 'Subject'],
      dtype='object')


# CatBoosting

In [8]:
def confusion_calcs(confusion_matrix):
    temp_arr = confusion_matrix
    tn = temp_arr[0]
    fp = temp_arr[1]
    fn = temp_arr[2]
    tp = temp_arr[3]
    sensitivity = tp / (tp + fn)
    recall = sensitivity
    specificity = tn / (tn + fp)
    precision = tp / (tp + fp)
    accuracy = (tp+tn) / (tp+tn+fn+fp)
    f1 = 2*precision*recall/(precision+recall)
    return [accuracy, sensitivity, specificity, precision, f1]

In [9]:
from sklearn.metrics import roc_curve, auc

In [ ]:
#Ignore the Rest of the STuff. Main One

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

model = CatBoostClassifier(learning_rate=0.1, iterations=100, depth=5, loss_function='Logloss', random_seed = 0)

#acc_score = [];
#Truth = [];
#Output = [];
iterations_list = []
#max_depth_list = []
#x_values = ['StoS']
# 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#        'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'

y_values = ['Rhythm']
#for df in features_df_list:
df = features_maindf



all_features = ['AppEn', 'SampEn', 'CosEn', 'RangeEn', 'FuzzyEn',
       'EntropyAF', 'PermEn', 'BubbleEn', 'MultiscaleEn', 'Volatility']
dataScores = [[],[],[],[],[],[],[],[],[],[]]
        
for score_idx,temp in tqdm(enumerate(all_features)):    
    
    x_values = [temp]
    print(x_values)

    acc_score = [];
    Truth = [];
    Output = [];



    for idx, subject in enumerate(good_list):

        train_index = ~df['Subject'].str.contains(subject)
        test_index = df['Subject'].str.contains(subject)
        X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
        y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values].values.ravel()

        model.fit(X_train, y_train)

        # make the prediction using the resulting model
        pred_values = model.predict(X_test)
        #preds_proba = model.predict_proba(test_data)
        #print("class = ", preds_class)
        #print("proba = ", preds_proba)

        acc = accuracy_score(y_test, pred_values)
        acc_score.append(acc)

        Truth.extend(y_test.reshape(y_test.shape[0])); ## it is a list
        Output.extend(pred_values); ## it is a list  

        #print(model.feature_importances_)
    #     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
    #                           index = (X_train.columns))
    #     Importance.sort_values(by = 'Importance', 
    #                        axis = 0, 
    #                        ascending = True).plot(kind = 'barh', 
    #                                               color = 'r')
    #     plt.xlabel('Variable Importance')
    #     plt.gca().legend_ = None
        #plt.savefig('plot1.png')


    elapsed = timeit.default_timer() - start_time #gives total computation time
    print("---Run time is %s seconds ---" % elapsed) #prints computation time
    print()

    print('Accuracy of each fold: \n {}'.format(acc_score))
    print()
    try: 
        print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
    except: 
        print('hi')
    print()
    print('Std of accuracy : \n{}'.format(np.std(acc_score)))
    print()
    print(confusion_matrix(Truth, Output))
    print()
    print(classification_report(Truth, Output))
    print()
    print(acc_score)
    print()
    print(elapsed)
    
    temp_array = confusion_matrix(Truth, Output)
    joint = temp_array[0].tolist() + temp_array[1].tolist()
    temp_scores = confusion_calcs(joint)
    
    #dataScores[score_idx].append(all_features[score_idx])
    #dataScores[score_idx].append(acc_score)
    dataScores[score_idx].append(np.mean(acc_score))
    dataScores[score_idx].append(np.std(acc_score))
    
    for confusionScore in temp_scores:
        dataScores[score_idx].append(confusionScore)

0it [00:00, ?it/s]

['AppEn']
0:	learn: 0.6261793	total: 158ms	remaining: 15.6s
1:	learn: 0.5712487	total: 162ms	remaining: 7.95s
2:	learn: 0.5257322	total: 166ms	remaining: 5.36s
3:	learn: 0.4872358	total: 170ms	remaining: 4.07s
4:	learn: 0.4544423	total: 173ms	remaining: 3.28s
5:	learn: 0.4263064	total: 176ms	remaining: 2.76s
6:	learn: 0.4020816	total: 179ms	remaining: 2.38s
7:	learn: 0.3812622	total: 182ms	remaining: 2.09s
8:	learn: 0.3633115	total: 185ms	remaining: 1.87s
9:	learn: 0.3475403	total: 188ms	remaining: 1.69s
10:	learn: 0.3338786	total: 191ms	remaining: 1.54s
11:	learn: 0.3216928	total: 194ms	remaining: 1.42s
12:	learn: 0.3109779	total: 197ms	remaining: 1.32s
13:	learn: 0.3019167	total: 200ms	remaining: 1.23s
14:	learn: 0.2936147	total: 203ms	remaining: 1.15s
15:	learn: 0.2862268	total: 206ms	remaining: 1.08s
16:	learn: 0.2797141	total: 209ms	remaining: 1.02s
17:	learn: 0.2740216	total: 212ms	remaining: 966ms
18:	learn: 0.2689845	total: 215ms	remaining: 916ms
19:	learn: 0.2646640	total: 218

62:	learn: 0.2338139	total: 186ms	remaining: 109ms
63:	learn: 0.2336861	total: 189ms	remaining: 106ms
64:	learn: 0.2336142	total: 192ms	remaining: 103ms
65:	learn: 0.2335605	total: 195ms	remaining: 100ms
66:	learn: 0.2335158	total: 197ms	remaining: 97.2ms
67:	learn: 0.2334070	total: 200ms	remaining: 94.3ms
68:	learn: 0.2333776	total: 203ms	remaining: 91.3ms
69:	learn: 0.2333330	total: 206ms	remaining: 88.3ms
70:	learn: 0.2333062	total: 209ms	remaining: 85.4ms
71:	learn: 0.2332784	total: 212ms	remaining: 82.3ms
72:	learn: 0.2332652	total: 214ms	remaining: 79.3ms
73:	learn: 0.2332302	total: 217ms	remaining: 76.4ms
74:	learn: 0.2331948	total: 221ms	remaining: 73.6ms
75:	learn: 0.2331393	total: 224ms	remaining: 70.7ms
76:	learn: 0.2331262	total: 227ms	remaining: 67.7ms
77:	learn: 0.2330639	total: 230ms	remaining: 64.8ms
78:	learn: 0.2330481	total: 232ms	remaining: 61.8ms
79:	learn: 0.2330051	total: 235ms	remaining: 58.8ms
80:	learn: 0.2329748	total: 238ms	remaining: 55.8ms
81:	learn: 0.232

60:	learn: 0.2155673	total: 184ms	remaining: 117ms
61:	learn: 0.2154780	total: 186ms	remaining: 114ms
62:	learn: 0.2153815	total: 190ms	remaining: 111ms
63:	learn: 0.2153246	total: 193ms	remaining: 108ms
64:	learn: 0.2153002	total: 196ms	remaining: 106ms
65:	learn: 0.2152270	total: 199ms	remaining: 103ms
66:	learn: 0.2151629	total: 202ms	remaining: 99.4ms
67:	learn: 0.2151099	total: 205ms	remaining: 96.3ms
68:	learn: 0.2150710	total: 208ms	remaining: 93.3ms
69:	learn: 0.2150476	total: 210ms	remaining: 90.2ms
70:	learn: 0.2149937	total: 213ms	remaining: 87.1ms
71:	learn: 0.2149710	total: 216ms	remaining: 84ms
72:	learn: 0.2149191	total: 219ms	remaining: 80.9ms
73:	learn: 0.2148768	total: 222ms	remaining: 77.9ms
74:	learn: 0.2148612	total: 225ms	remaining: 74.9ms
75:	learn: 0.2147791	total: 228ms	remaining: 71.9ms
76:	learn: 0.2147586	total: 230ms	remaining: 68.8ms
77:	learn: 0.2146840	total: 233ms	remaining: 65.7ms
78:	learn: 0.2146643	total: 236ms	remaining: 62.7ms
79:	learn: 0.2146521

59:	learn: 0.2339336	total: 176ms	remaining: 117ms
60:	learn: 0.2338971	total: 178ms	remaining: 114ms
61:	learn: 0.2338705	total: 181ms	remaining: 111ms
62:	learn: 0.2337148	total: 184ms	remaining: 108ms
63:	learn: 0.2336355	total: 187ms	remaining: 105ms
64:	learn: 0.2335764	total: 190ms	remaining: 102ms
65:	learn: 0.2335313	total: 193ms	remaining: 99.3ms
66:	learn: 0.2334834	total: 195ms	remaining: 96.2ms
67:	learn: 0.2333806	total: 198ms	remaining: 93.2ms
68:	learn: 0.2333329	total: 201ms	remaining: 90.4ms
69:	learn: 0.2333037	total: 204ms	remaining: 87.3ms
70:	learn: 0.2332611	total: 207ms	remaining: 84.4ms
71:	learn: 0.2332331	total: 209ms	remaining: 81.4ms
72:	learn: 0.2332115	total: 212ms	remaining: 78.4ms
73:	learn: 0.2331947	total: 215ms	remaining: 75.4ms
74:	learn: 0.2331787	total: 217ms	remaining: 72.5ms
75:	learn: 0.2331256	total: 220ms	remaining: 69.6ms
76:	learn: 0.2331115	total: 223ms	remaining: 66.6ms
77:	learn: 0.2330530	total: 226ms	remaining: 63.6ms
78:	learn: 0.23304

...

88:	learn: 0.3396014	total: 266ms	remaining: 32.9ms
89:	learn: 0.3395928	total: 269ms	remaining: 29.9ms
90:	learn: 0.3395846	total: 272ms	remaining: 26.9ms
91:	learn: 0.3395775	total: 275ms	remaining: 23.9ms
92:	learn: 0.3395682	total: 279ms	remaining: 21ms
93:	learn: 0.3395517	total: 282ms	remaining: 18ms
94:	learn: 0.3395442	total: 285ms	remaining: 15ms
95:	learn: 0.3395309	total: 289ms	remaining: 12ms
96:	learn: 0.3395285	total: 292ms	remaining: 9.02ms
97:	learn: 0.3395229	total: 295ms	remaining: 6.02ms
98:	learn: 0.3395108	total: 298ms	remaining: 3.01ms
99:	learn: 0.3394902	total: 301ms	remaining: 0us
0:	learn: 0.6380645	total: 5.27ms	remaining: 522ms
1:	learn: 0.5930889	total: 9.15ms	remaining: 448ms
2:	learn: 0.5560926	total: 12.9ms	remaining: 417ms
3:	learn: 0.5253017	total: 16ms	remaining: 384ms
4:	learn: 0.4993011	total: 19.4ms	remaining: 369ms
5:	learn: 0.4769992	total: 22.7ms	remaining: 355ms
6:	learn: 0.4583723	total: 25.9ms	remaining: 345ms
7:	learn: 0.4423034	total: 29.1m

74:	learn: 0.3026856	total: 240ms	remaining: 80ms
75:	learn: 0.3026787	total: 243ms	remaining: 76.9ms
76:	learn: 0.3026745	total: 247ms	remaining: 73.7ms
77:	learn: 0.3026685	total: 250ms	remaining: 70.6ms
78:	learn: 0.3026646	total: 254ms	remaining: 67.5ms
79:	learn: 0.3026610	total: 257ms	remaining: 64.3ms
80:	learn: 0.3026553	total: 261ms	remaining: 61.2ms
81:	learn: 0.3026493	total: 264ms	remaining: 57.9ms
82:	learn: 0.3026471	total: 267ms	remaining: 54.7ms
83:	learn: 0.3026452	total: 270ms	remaining: 51.5ms
84:	learn: 0.3026404	total: 273ms	remaining: 48.3ms
85:	learn: 0.3026328	total: 277ms	remaining: 45ms
86:	learn: 0.3026256	total: 280ms	remaining: 41.9ms
87:	learn: 0.3026210	total: 284ms	remaining: 38.7ms
88:	learn: 0.3026115	total: 287ms	remaining: 35.5ms
89:	learn: 0.3026035	total: 291ms	remaining: 32.3ms
90:	learn: 0.3025968	total: 294ms	remaining: 29.1ms
91:	learn: 0.3025941	total: 297ms	remaining: 25.8ms
92:	learn: 0.3025887	total: 300ms	remaining: 22.6ms
93:	learn: 0.302

59:	learn: 0.3375774	total: 180ms	remaining: 120ms
60:	learn: 0.3375543	total: 184ms	remaining: 117ms
61:	learn: 0.3375301	total: 187ms	remaining: 115ms
62:	learn: 0.3375087	total: 190ms	remaining: 111ms
63:	learn: 0.3374880	total: 193ms	remaining: 108ms
64:	learn: 0.3374778	total: 196ms	remaining: 106ms
65:	learn: 0.3374535	total: 199ms	remaining: 103ms
66:	learn: 0.3374315	total: 202ms	remaining: 99.6ms
67:	learn: 0.3374185	total: 206ms	remaining: 96.7ms
68:	learn: 0.3374094	total: 209ms	remaining: 93.8ms
69:	learn: 0.3374044	total: 212ms	remaining: 90.9ms
70:	learn: 0.3373860	total: 215ms	remaining: 88ms
71:	learn: 0.3373744	total: 219ms	remaining: 85.1ms
72:	learn: 0.3373698	total: 222ms	remaining: 82.1ms
73:	learn: 0.3373631	total: 225ms	remaining: 79.2ms
74:	learn: 0.3373546	total: 228ms	remaining: 76.2ms
75:	learn: 0.3373513	total: 232ms	remaining: 73.2ms
76:	learn: 0.3373365	total: 235ms	remaining: 70.2ms
77:	learn: 0.3373188	total: 238ms	remaining: 67.2ms
78:	learn: 0.3372984	

59:	learn: 0.3283168	total: 187ms	remaining: 125ms
60:	learn: 0.3283087	total: 191ms	remaining: 122ms
61:	learn: 0.3282862	total: 194ms	remaining: 119ms
62:	learn: 0.3282756	total: 197ms	remaining: 116ms
63:	learn: 0.3282551	total: 200ms	remaining: 113ms
64:	learn: 0.3282428	total: 204ms	remaining: 110ms
65:	learn: 0.3282231	total: 207ms	remaining: 106ms
66:	learn: 0.3282024	total: 210ms	remaining: 103ms
67:	learn: 0.3281861	total: 213ms	remaining: 100ms
68:	learn: 0.3281773	total: 215ms	remaining: 96.8ms
69:	learn: 0.3281630	total: 218ms	remaining: 93.5ms
70:	learn: 0.3281582	total: 221ms	remaining: 90.4ms
71:	learn: 0.3281492	total: 224ms	remaining: 87.2ms
72:	learn: 0.3281331	total: 227ms	remaining: 84ms
73:	learn: 0.3281223	total: 230ms	remaining: 80.9ms
74:	learn: 0.3281170	total: 233ms	remaining: 77.7ms
75:	learn: 0.3281094	total: 236ms	remaining: 74.5ms
76:	learn: 0.3280992	total: 239ms	remaining: 71.4ms
77:	learn: 0.3280768	total: 242ms	remaining: 68.3ms
78:	learn: 0.3280630	to

61:	learn: 0.3344263	total: 186ms	remaining: 114ms
62:	learn: 0.3343991	total: 189ms	remaining: 111ms
63:	learn: 0.3343770	total: 193ms	remaining: 108ms
64:	learn: 0.3343670	total: 196ms	remaining: 105ms
65:	learn: 0.3343529	total: 199ms	remaining: 103ms
66:	learn: 0.3343328	total: 202ms	remaining: 99.6ms
67:	learn: 0.3343143	total: 206ms	remaining: 96.7ms
68:	learn: 0.3343083	total: 209ms	remaining: 93.9ms
69:	learn: 0.3342960	total: 212ms	remaining: 90.8ms
70:	learn: 0.3342848	total: 215ms	remaining: 87.9ms
71:	learn: 0.3342641	total: 218ms	remaining: 84.9ms
72:	learn: 0.3342567	total: 222ms	remaining: 82ms
73:	learn: 0.3342470	total: 225ms	remaining: 79ms
74:	learn: 0.3342358	total: 228ms	remaining: 76ms
75:	learn: 0.3342343	total: 231ms	remaining: 72.9ms
76:	learn: 0.3342308	total: 234ms	remaining: 69.9ms
77:	learn: 0.3342219	total: 237ms	remaining: 66.9ms
78:	learn: 0.3342123	total: 241ms	remaining: 64ms
79:	learn: 0.3341821	total: 244ms	remaining: 61ms
80:	learn: 0.3341747	total:

In [ ]:
    
export_df = pd.DataFrame(dataScores,columns=['Mean Acc.', 'SD of Acc.', 'Total Acc.', 'Sensitivity', 
                                             'Specificity','Precision','f1'])

export_df = export_df.set_index(pd.Index(['AppEn', 'SampEn', 'CosEn', 'RangeEn', 'FuzzyEn',
       'EntropyAF', 'PermEn', 'BubbleEn', 'MultiscaleEn', 'Volatility']))

In [ ]:
export_df

In [17]:
export_df_highlight = export_df.copy()

export_df_highlight.style.highlight_max(color = 'lightgreen', axis = 0)

,Mean Acc.,SD of Acc.,Total Acc.,Sensitivity,Specificity,Precision,f1
AppEn,0.882025,0.159679,0.870619,0.864129,0.875925,0.850586,0.857304
SampEn,0.879405,0.160325,0.869952,0.891125,0.852645,0.831741,0.860410
CosEn,0.879405,0.160325,0.869952,0.891125,0.852645,0.831741,0.860410
RangeEn,0.880318,0.098054,0.879803,0.903782,0.860203,0.840878,0.871196
FuzzyEn,0.771909,0.119450,0.772594,0.826502,0.728529,0.713353,0.765770
EntropyAF,0.863567,0.141360,0.864793,0.913572,0.824920,0.810075,0.858716
PermEn,0.631511,0.113435,0.637765,0.606329,0.663460,0.595580,0.600907
BubbleEn,0.616064,0.105295,0.618484,0.550260,0.674251,0.579968,0.564723
MultiscaleEn,0.855645,0.162571,0.847536,0.856564,0.840157,0.814136,0.834811
Volatility,0.780561,0.203759,0.779732,0.915797,0.668512,0.693085,0.789026


In [18]:
export_df_highlight.style.highlight_min(color = 'lightcoral', axis = 0)

,Mean Acc.,SD of Acc.,Total Acc.,Sensitivity,Specificity,Precision,f1
AppEn,0.882025,0.159679,0.870619,0.864129,0.875925,0.850586,0.857304
SampEn,0.879405,0.160325,0.869952,0.891125,0.852645,0.831741,0.860410
CosEn,0.879405,0.160325,0.869952,0.891125,0.852645,0.831741,0.860410
RangeEn,0.880318,0.098054,0.879803,0.903782,0.860203,0.840878,0.871196
FuzzyEn,0.771909,0.119450,0.772594,0.826502,0.728529,0.713353,0.765770
EntropyAF,0.863567,0.141360,0.864793,0.913572,0.824920,0.810075,0.858716
PermEn,0.631511,0.113435,0.637765,0.606329,0.663460,0.595580,0.600907
BubbleEn,0.616064,0.105295,0.618484,0.550260,0.674251,0.579968,0.564723
MultiscaleEn,0.855645,0.162571,0.847536,0.856564,0.840157,0.814136,0.834811
Volatility,0.780561,0.203759,0.779732,0.915797,0.668512,0.693085,0.789026


In [19]:
export_df.to_csv(f'new-individual-features/CatBoost.csv')

In [15]:
#Ignore Stuff Below

In [16]:
#Ignore Stuff Below

In [17]:
#Ignore Stuff Below

In [18]:
#Ignore Stuff Below

In [19]:
#Ignore Stuff Below

In [20]:
#Ignore Stuff Below